# Generating text using Recurrent Neural Network

## Preprocessing data

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()

In [ ]:
with open('data/irish_song.txt', 'r') as f:
    corpus = f.read().lower().splitlines()


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1


In [ ]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[ :i+1]
        input_sequences.append(n_gram_sequence)
        

### Padding

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
import numpy as np

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


### Splitting the last word from the rest of the sentence

In [ ]:
xs = input_sequences[ : , :-1]
labels = input_sequences[ : , -1]

### Convert label (ys) to be categorical and One Hot Encoded

In [ ]:
import tensorflow

In [ ]:
ys = tensorflow.keras.utils.to_categorical(labels, num_classes=total_words)

# Building the RNN

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)



In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
history = model.fit(xs, ys, epochs=25, verbose=1)

### Printing model summary

In [ ]:
print(model.summary())

### Plots

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')

### Generating text

In [ ]:
seed_text = "I've got a bad feeling about this"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)